# Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, project_tests
%autoreload 1

2024-09-15 13:58:20.519921: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-15 13:58:20.659800: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-15 13:58:20.659829: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-15 13:58:20.659893: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-15 13:58:20.689916: I tensorflow/core/platform/cpu_feature_g

In [ ]:
# Should return keras==2.14.0 and tf-keras==2.14.1
%pip freeze | grep keras

/bin/bash: /home/oliver/anaconda3/envs/tf-2.14-gpu/lib/libtinfo.so.6: no version information available (required by /bin/bash)
keras==2.14.0


In [ ]:
# Should return tensorflow==2.14.0
%pip freeze | grep tensorflow

/bin/bash: /home/oliver/anaconda3/envs/tf-2.14-gpu/lib/libtinfo.so.6: no version information available (required by /bin/bash)
tensorflow==2.14.0
tensorflow-datasets==4.9.3
tensorflow-estimator==2.14.0
tensorflow-hub==0.15.0
tensorflow-io-gcs-filesystem==0.37.1
tensorflow-metadata==1.15.0


In [12]:
import collections

import helper
import numpy as np
import project_tests as tests

from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from tensorflow.keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.utils import plot_model

## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [5]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [6]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [7]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [8]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    x_tk = Tokenizer(char_level=False)
    x_tk.fit_on_texts(x)

    return x_tk.texts_to_sequences(x), x_tk
    
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)

for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [9]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen=length, padding='post')
    
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [10]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [11]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)

A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

<p align="center">
    <img src="./model_simple_plot.png" alt="Architecture of Encoder Decoder" />
</p>

In [15]:
from tensorflow.keras.layers import Input, SimpleRNN, Dense, TimeDistributed, Activation
import sys

def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence (not used in this simple model version)
    :param english_vocab_size: Number of unique English words in the dataset (not used in this simple model version)
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 1e-3
  
    # Input layer: sequences of English words (integers)
    input_seq = Input(shape=input_shape[1:]) # we remove the batch size, because Keras infers it implicitely (Caution: it is a 1d tuple, not a numpy array)
    
    # Simple RNN layer (return_sequences=True to output the entire sequence)
    rnn = SimpleRNN(64, return_sequences=True)(input_seq)
    
    # Fully connected layer with output for each time step
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    
    # Apply softmax to output a probability distribution over the French vocabulary
    output_seq = Activation('softmax')(logits)
    
    # Create the model
    model = Model(inputs=input_seq, outputs=output_seq)
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
    
tests.test_simple_model(simple_model)

# Important: we prepare the english sentences so that it pads them to the size of the target language (french),
# because the RNN returns all processed english sentences and feeds that to the TimeDistributed DenseLayer, 
# and that needs the same input size as output size because of the nature of sequentual processing.
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)

# Reshaping the input to work with a basic RNN (RNN needs a third dimension for sequencual processing]
tmp_x = tmp_x.reshape((tmp_x.shape[0], tmp_x.shape[1], 1))  # (batch_size, sequence_length, 1)

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

# Plot the model and save it as an image
plot_model(simple_rnn_model, to_file='./model_simple_plot.png', show_shapes=True, show_layer_names=True)

simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10


2024-09-15 14:02:26.931304: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f9a580144a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-09-15 14:02:26.931344: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-09-15 14:02:26.941165: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-15 14:02:26.968324: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2024-09-15 14:02:27.070008: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


108/108 [==============================] - 4s 26ms/step - loss: 3.3433 - accuracy: 0.4070 - val_loss: nan - val_accuracy: 0.4589
Epoch 2/10
108/108 [==============================] - 2s 22ms/step - loss: 2.4171 - accuracy: 0.4693 - val_loss: nan - val_accuracy: 0.4915
Epoch 3/10
108/108 [==============================] - 2s 22ms/step - loss: 2.1892 - accuracy: 0.5185 - val_loss: nan - val_accuracy: 0.5305
Epoch 4/10
108/108 [==============================] - 2s 23ms/step - loss: 2.0133 - accuracy: 0.5419 - val_loss: nan - val_accuracy: 0.5554
Epoch 5/10
108/108 [==============================] - 2s 22ms/step - loss: 1.8704 - accuracy: 0.5586 - val_loss: nan - val_accuracy: 0.5654
Epoch 6/10
108/108 [==============================] - 2s 22ms/step - loss: 1.7618 - accuracy: 0.5729 - val_loss: nan - val_accuracy: 0.5795
Epoch 7/10
108/108 [==============================] - 2s 22ms/step - loss: 1.6861 - accuracy: 0.5824 - val_loss: nan - val_accuracy: 0.5859
Epoch 8/10
108/108 [===========

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)

You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

<p align="center">
    <img src="./model_embed_plot.png" alt="Architecture of Encoder Decoder" />
</p>

In [18]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    learning_rate = 1e-3
    
    # Input layer: sequences of English words (integers)
    input_seq = Input(shape=input_shape[1:]) # we remove the batch size, because Keras infers it implicitely

    encoder_embedding = Embedding(input_dim=english_vocab_size, output_dim=100, mask_zero=False)(input_seq)
    
    # Simple RNN layer (return_sequences=True to output the entire sequence)
    rnn = SimpleRNN(64, return_sequences=True)(encoder_embedding)
    
    # Fully connected layer with output for each time step
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    
    # Apply softmax to output a probability distribution over the French vocabulary
    output_seq = Activation('softmax')(logits)
    
    # Create the model
    model = Model(inputs=input_seq, outputs=output_seq)
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
    
tests.test_embed_model(embed_model)

# Reshaping the input to work with an RNN with an embedding
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, max_french_sequence_length))

# Train the neural network
embed_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

# Plot the model and save it as an image
plot_model(embed_model, to_file='./model_embed_plot.png', show_shapes=True, show_layer_names=True)

embed_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(embed_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
108/108 [==============================] - 6s 49ms/step - loss: 3.5013 - accuracy: 0.4067 - val_loss: nan - val_accuracy: 0.4561
Epoch 2/10
108/108 [==============================] - 4s 36ms/step - loss: 2.2968 - accuracy: 0.5227 - val_loss: nan - val_accuracy: 0.5900
Epoch 3/10
108/108 [==============================] - 3s 32ms/step - loss: 1.5914 - accuracy: 0.6426 - val_loss: nan - val_accuracy: 0.6820
Epoch 4/10
108/108 [==============================] - 4s 33ms/step - loss: 1.2088 - accuracy: 0.7154 - val_loss: nan - val_accuracy: 0.7471
Epoch 5/10
108/108 [==============================] - 4s 33ms/step - loss: 0.9625 - accuracy: 0.7632 - val_loss: nan - val_accuracy: 0.7794
Epoch 6/10
108/108 [==============================] - 4s 33ms/step - loss: 0.8126 - accuracy: 0.7931 - val_loss: nan - val_accuracy: 0.8053
Epoch 7/10
108/108 [==============================] - 4s 34ms/step - loss: 0.7109 - accuracy: 0.8159 - val_loss: nan - val_accuracy: 0.8250
Epoch 8/10
108/108 [

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

<p align="center">
    <img src="./model_birnn_plot.png" alt="Architecture of Encoder Decoder" />
</p>

In [19]:
from tensorflow.keras.layers import Input, SimpleRNN, Bidirectional, Dense, TimeDistributed, Activation, Embedding

def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    learning_rate = 1e-3
    hidden_units = 128
    
    # Input layer: sequences of English words (integers)
    input_seq = Input(shape=input_shape[1:]) # we remove the batch size, because Keras infers it implicitely

    # Bidirectional RNN
    rnn = Bidirectional(
        SimpleRNN(hidden_units, return_sequences=True, return_state=False),
        name='bidirectional_encoder'
    )(input_seq)

    # Fully connected layer with output for each time step
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    
    # Apply softmax to output a probability distribution over the French vocabulary
    output_seq = Activation('softmax')(logits)
    
    # Create the model
    model = Model(inputs=input_seq, outputs=output_seq)
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model    
    
tests.test_bd_model(bd_model)

# Reshaping the input to work with an RNN with an embedding
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)

# The bidirectional model is without embedding (I assume so because of the test_bd_model), so the input needs a third dimension for the tokens
tmp_x = tmp_x.reshape((tmp_x.shape[0], tmp_x.shape[1], 1))  # (batch_size, sequence_length, 1)

# Train the neural network
bd_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

# Plot the model and save it as an image
plot_model(bd_model, to_file='./model_birnn_plot.png', show_shapes=True, show_layer_names=True)

bd_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bd_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
108/108 [==============================] - 7s 49ms/step - loss: 2.4582 - accuracy: 0.5130 - val_loss: nan - val_accuracy: 0.5854
Epoch 2/10
108/108 [==============================] - 5s 45ms/step - loss: 1.6508 - accuracy: 0.5984 - val_loss: nan - val_accuracy: 0.6131
Epoch 3/10
108/108 [==============================] - 5s 45ms/step - loss: 1.4637 - accuracy: 0.6178 - val_loss: nan - val_accuracy: 0.6268
Epoch 4/10
108/108 [==============================] - 5s 43ms/step - loss: 1.3641 - accuracy: 0.6322 - val_loss: nan - val_accuracy: 0.6377
Epoch 5/10
108/108 [==============================] - 5s 44ms/step - loss: 1.2904 - accuracy: 0.6448 - val_loss: nan - val_accuracy: 0.6517
Epoch 6/10
108/108 [==============================] - 5s 48ms/step - loss: 1.2329 - accuracy: 0.6568 - val_loss: nan - val_accuracy: 0.6631
Epoch 7/10
108/108 [==============================] - 5s 44ms/step - loss: 1.1857 - accuracy: 0.6651 - val_loss: nan - val_accuracy: 0.6695
Epoch 8/10
108/108 [

### Model 4: Encoder-Decoder (IMPLEMENTATION)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [21]:
from tensorflow.keras.layers import Input, SimpleRNN, Dense, Activation, Embedding, Bidirectional, Concatenate, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.utils import plot_model
import tensorflow.keras.backend as K
import tensorflow as tf

def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic encoder-decoder RNN for machine translation without teacher forcing.
    :param input_shape: Tuple of input shape (input sequence)
    :param output_sequence_length: Length of output sequence (target sequence)
    :param english_vocab_size: Number of unique English words in the dataset (input)
    :param french_vocab_size: Number of unique French words in the dataset (output)
    :return: Keras model built, but not trained
    """
    learning_rate = 1e-3
    hidden_units = 128
    
    ### ENCODER
    
    # Input layer: sequences of English words (tokenized integers)
    encoder_input = Input(shape=input_shape[1:])  # Shape (sequence_length,)    
    
    # Simple RNN for the encoder (using the final hidden state)
    _, encoder_hidden = SimpleRNN(hidden_units, return_sequences=False, return_state=True)(encoder_input)        
    
    ### DECODER
    
    # So this here is special, instead of solely rely on the context vector we feed the previous generated word 
    # as input to the next time sequence. For that I have to implement the loop by myself because Keras doesn't support
    # this out of the box.
    
    decoder_input = tf.zeros((tf.shape(encoder_input)[0], 1, 1), dtype=tf.float32)
    
    # Define the RNN cell and Dense layer outside the loop
    decoder_rnn_cell = SimpleRNN(hidden_units, return_state=True, name='decoder_rnn_cell')
    decoder_dense = Dense(french_vocab_size, activation='softmax', name='decoder_output_dense')

    # Initialize variables for the loop
    outputs = []
    state = encoder_hidden

    # Loop over the output sequence length
    for _ in range(output_sequence_length):
        # Convert the integer tokens to float32 for compatibility with RNN layers (RNNs require float inputs)
        decoder_input_float = tf.cast(decoder_input, tf.float32)        
        
        # Run one step of the decoder RNN
        decoder_output, state = decoder_rnn_cell(decoder_input_float, initial_state=state)
    
        # Compute the output probabilities
        decoder_logits = decoder_dense(decoder_output)  # Shape: (batch_size, french_vocab_size)
    
        # Append the output
        outputs.append(tf.expand_dims(decoder_logits, 1))  # Shape: (batch_size, 1, french_vocab_size)
    
        # Use the last predicted word as the next input (greedy decoding)
        predicted_token = tf.argmax(decoder_logits, axis=-1)  # Shape: (batch_size,)
        
        # Update the decoder input for the next time step (use predicted tokens)
        decoder_input = tf.expand_dims(tf.expand_dims(predicted_token, axis=1), axis=-1)
    
    # Concatenate all outputs
    decoder_outputs = Concatenate(axis=1, name='decoder_outputs')(outputs)  # Shape: (batch_size, output_sequence_length, french_vocab_size)

    # Create the model
    model = Model(inputs=encoder_input, outputs=decoder_outputs)
    
    # Compile the model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_encdec_model(encdec_model)

# Reshape the input English sentences (padding to max length of French sentences)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)

# Reshape to match the input shape requirements (add the feature dimension)
tmp_x = tmp_x.reshape((tmp_x.shape[0], tmp_x.shape[1], 1))  # (batch_size, sequence_length, 1)

# Set the correct input shape and vocabulary sizes
input_shape = tmp_x.shape
output_sequence_length = preproc_french_sentences.shape[1]

# Remove the last dimension (reshape to (batch_size, sequence_length))
preproc_french_sentences = preproc_french_sentences.reshape((-1, preproc_french_sentences.shape[1]))

# Build the encoder-decoder model
encoder_decoder_rnn_model = encdec_model(
    input_shape,
    output_sequence_length,
    english_vocab_size,
    french_vocab_size
)

# Plot the model and save it as an image
plot_model(encoder_decoder_rnn_model, to_file='./model_encdec_plot.png', show_shapes=True, show_layer_names=True)

# Train the model using English sentences as input and French sentences as labels (as before)
encoder_decoder_rnn_model.fit(tmp_x, preproc_french_sentences, 
                              batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(encoder_decoder_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
108/108 [==============================] - 17s 71ms/step - loss: 3.3714 - accuracy: 0.4054 - val_loss: nan - val_accuracy: 0.4713
Epoch 2/10
108/108 [==============================] - 6s 56ms/step - loss: 2.3189 - accuracy: 0.5023 - val_loss: nan - val_accuracy: 0.5394
Epoch 3/10
108/108 [==============================] - 6s 55ms/step - loss: 1.9649 - accuracy: 0.5506 - val_loss: nan - val_accuracy: 0.5651
Epoch 4/10
108/108 [==============================] - 6s 54ms/step - loss: 1.7142 - accuracy: 0.5819 - val_loss: nan - val_accuracy: 0.5928
Epoch 5/10
108/108 [==============================] - 6s 54ms/step - loss: 1.5614 - accuracy: 0.5989 - val_loss: nan - val_accuracy: 0.6067
Epoch 6/10
108/108 [==============================] - 6s 56ms/step - loss: 1.4623 - accuracy: 0.6123 - val_loss: nan - val_accuracy: 0.6151
Epoch 7/10
108/108 [==============================] - 6s 56ms/step - loss: 1.4201 - accuracy: 0.6170 - val_loss: nan - val_accuracy: 0.6155
Epoch 8/10
108/108 

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [27]:
from tensorflow.keras.layers import Input, SimpleRNN, Dense, Activation, Embedding, Bidirectional, Concatenate, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy
import tensorflow as tf

def final_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    learning_rate = 1e-3
    hidden_units = 512
    
    ### ENCODER
    
    # Input layer: sequences of English words (tokenized integers)
    encoder_input = Input(shape=input_shape[1:], name='encoder_input')  # Shape: (sequence_length,)

    # Embedding layer for the encoder
    encoder_embedding = Embedding(input_dim=english_vocab_size, output_dim=hidden_units, name='encoder_embedding')(encoder_input)
    
    # Bidirectional RNN for the encoder
    encoder_outputs, forward_h, backward_h = Bidirectional(
        SimpleRNN(hidden_units, return_sequences=False, return_state=True),
        name='bidirectional_encoder'
    )(encoder_embedding)
    
    # Concatenate the forward and backward hidden states
    encoder_hidden = Concatenate(name='encoder_hidden')([forward_h, backward_h])  # Shape: (batch_size, 2 * hidden_units)
    
    # Optionally, transform encoder_hidden back to hidden_units size
    encoder_hidden = Dense(hidden_units, activation='tanh', name='encoder_hidden_dense')(encoder_hidden)
    
    ### DECODER
    
    # Initialize decoder input with the start token
    def create_initial_decoder_input(encoder_input):
        batch_size = tf.shape(encoder_input)[0]
        start_tokens = tf.fill([batch_size, 1], 0)  # Shape: (batch_size, 1)
        return start_tokens  # Integer tensor

    #decoder_input = Lambda(create_initial_decoder_input, name='decoder_input')(encoder_input)  # Shape: (batch_size, 1)
    decoder_input = create_initial_decoder_input(encoder_input)
    
    # Embedding layer for the decoder
    decoder_embedding_layer = Embedding(input_dim=french_vocab_size, output_dim=hidden_units, name='decoder_embedding')

    # So this here is special, instead of solely rely on the context vector we feed the previous generated word 
    # as input to the next time sequence. For that I have to implement the loop by myself because Keras doesn't support
    # this out of the box. 
    
    # Define the RNN cell and Dense layer outside the loop
    decoder_rnn_cell = SimpleRNN(hidden_units, return_state=True, name='decoder_rnn_cell')
    decoder_dense = Dense(french_vocab_size, activation='softmax', name='decoder_output_dense')
    
    # Initialize variables for the loop
    outputs = []
    state = encoder_hidden
    decoder_input_embedded = decoder_embedding_layer(decoder_input)  # Shape: (batch_size, 1, hidden_units)
    
    # Loop over the output sequence length
    for _ in range(output_sequence_length):
        # Run one step of the decoder RNN
        decoder_output, state = decoder_rnn_cell(decoder_input_embedded, initial_state=state)
        
        # Compute the output probabilities
        decoder_logits = decoder_dense(decoder_output)  # Shape: (batch_size, french_vocab_size)
        
        # Append the output
        outputs.append(tf.expand_dims(decoder_logits, 1))  # Shape: (batch_size, 1, french_vocab_size)
        
        # Use the last predicted word as the next input (greedy decoding)
        predicted_token = tf.argmax(decoder_logits, axis=-1)  # Shape: (batch_size,)
        decoder_input_embedded = decoder_embedding_layer(tf.expand_dims(predicted_token, 1))  # Shape: (batch_size, 1, hidden_units)
    
    # Concatenate all outputs
    decoder_outputs = Concatenate(axis=1, name='decoder_outputs')(outputs)  # Shape: (batch_size, output_sequence_length, french_vocab_size)
    
    # Create the model
    model = Model(inputs=encoder_input, outputs=decoder_outputs)
    
    # Compile the model
    model.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(learning_rate), metrics=['accuracy'])
    
    return model

tests.test_model_final(final_model)

# Reshape the input English sentences (padding to max length of French sentences)
tmp_x = pad(preproc_english_sentences, max_english_sequence_length)

# Set the correct input shape and vocabulary sizes
input_shape = tmp_x.shape
output_sequence_length = preproc_french_sentences.shape[1]

# Remove the last dimension (reshape to (batch_size, sequence_length))
preproc_french_sentences = preproc_french_sentences.reshape((-1, preproc_french_sentences.shape[1]))

# Build the final model
model_final = final_model(
    input_shape,
    output_sequence_length,
    english_vocab_size,
    french_vocab_size
)

# Plot the model and save it as an image
plot_model(model_final, to_file='./model_final_plot.png', show_shapes=True, show_layer_names=True)

# Train the model using English sentences as input and French sentences as labels (as before)
model_final.fit(tmp_x, preproc_french_sentences, 
                    batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(model_final.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
108/108 [==============================] - 30s 163ms/step - loss: 2.7260 - accuracy: 0.4602 - val_loss: nan - val_accuracy: 0.5407
Epoch 2/10
108/108 [==============================] - 13s 122ms/step - loss: 1.7397 - accuracy: 0.5876 - val_loss: nan - val_accuracy: 0.6069
Epoch 3/10
108/108 [==============================] - 13s 117ms/step - loss: 1.5275 - accuracy: 0.6021 - val_loss: nan - val_accuracy: 0.6133
Epoch 4/10
108/108 [==============================] - 13s 121ms/step - loss: 1.3566 - accuracy: 0.6323 - val_loss: nan - val_accuracy: 0.6440
Epoch 5/10
108/108 [==============================] - 13s 117ms/step - loss: 1.2995 - accuracy: 0.6433 - val_loss: nan - val_accuracy: 0.6531
Epoch 6/10
108/108 [==============================] - 13s 118ms/step - loss: 1.2761 - accuracy: 0.6493 - val_loss: nan - val_accuracy: 0.6509
Epoch 7/10
108/108 [==============================] - 13s 118ms/step - loss: 1.1699 - accuracy: 0.6713 - val_loss: nan - val_accuracy: 0.6771
Epoch 

## Prediction (IMPLEMENTATION)

In [26]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    # Reshape the input English sentences (padding to max length of english sentences)
    tmp_x = pad(x, max_english_sequence_length)
    
    # Set the correct input shape and vocabulary sizes
    input_shape = tmp_x.shape
    output_sequence_length = y.shape[1]
    
    # Remove the last dimension (reshape to (batch_size, sequence_length))
    preproc_french_sentences = y.reshape((-1, y.shape[1]))
    
    # Build the final model
    model = final_model(
        input_shape,
        output_sequence_length,
        english_vocab_size,
        french_vocab_size
    )
    
    # Train the model using English sentences as input and French sentences as labels (as before)
    model.fit(tmp_x, preproc_french_sentences, 
                        batch_size=1024, epochs=30, validation_split=0.2)

     
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))

final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Epoch 1/30
108/108 [==============================] - 28s 149ms/step - loss: 2.6838 - accuracy: 0.4481 - val_loss: nan - val_accuracy: 0.5224
Epoch 2/30
108/108 [==============================] - 14s 130ms/step - loss: 1.7029 - accuracy: 0.5911 - val_loss: nan - val_accuracy: 0.6239
Epoch 3/30
108/108 [==============================] - 14s 129ms/step - loss: 1.2618 - accuracy: 0.6608 - val_loss: nan - val_accuracy: 0.6740
Epoch 4/30
108/108 [==============================] - 13s 125ms/step - loss: 1.0937 - accuracy: 0.6950 - val_loss: nan - val_accuracy: 0.7117
Epoch 5/30
108/108 [==============================] - 13s 124ms/step - loss: 0.9811 - accuracy: 0.7219 - val_loss: nan - val_accuracy: 0.7329
Epoch 6/30
108/108 [==============================] - 13s 122ms/step - loss: 0.8890 - accuracy: 0.7445 - val_loss: nan - val_accuracy: 0.7572
Epoch 7/30
108/108 [==============================] - 13s 122ms/step - loss: 0.8027 - accuracy: 0.7691 - val_loss: nan - val_accuracy: 0.7814
Epoch 

## Submission
When you're ready to submit, complete the following steps:
1. Review the rubric to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [ ]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?